In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AdamW

# Load the dataset
df = pd.read_csv("hausa.csv")

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the text data with right padding
train_encodings = tokenizer(
    list(train_df["text"]),
    truncation=True,
    padding=True,
    max_length=32,
    return_tensors="pt"
)

val_encodings = tokenizer(
    list(val_df["text"]),
    truncation=True,
    padding=True,
    max_length=32,
    return_tensors="pt"
)

# Convert the labels to categorical values
train_labels = torch.tensor(train_df["label"].map({"negative": 0, "positive": 1}).values)
val_labels = torch.tensor(val_df["label"].map({"negative": 0, "positive": 1}).values)

# Define the model and optimizer
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,
    save_total_limit=2,
    save_steps=500,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_strategy="steps",
    save_on_each_node=True,
)

# Define the Trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,  # Use train_encodings here
    eval_dataset=val_encodings,      # Use val_encodings here
    compute_metrics=lambda pred: {"accuracy": accuracy_score(val_labels, pred.predictions.argmax(1))},
)
trainer.train()


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/isab7070/miniconda3/envs/mynlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/isab7070/miniconda3/envs/mynlp/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/3 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,labels,output_attentions,output_hidden_states,return_dict,label_ids,labels,label.